In [3]:
# !pip install quantstats

In [4]:

import polars as pl

# df = pl.read_csv('sample_portfolio.csv')
df = pl.read_csv('sample_portfolio_acn3.csv')

df = df.sort(['tr_id', 'rdate'])
df = df.with_columns(pl.col('sec_price').shift().over('tr_id').alias('prev_sec_price'))

df = df.with_columns([
    pl.col('tr_shares').cumsum().over('tr_id').alias('position'),
    (pl.col('tr_shares') * pl.col('sec_price')).fill_null(0).cumsum().over('tr_id').alias('cost_basis'),
    ((pl.col('sec_price') - pl.col('prev_sec_price')) / pl.col('prev_sec_price') * 100).fill_null(0).alias('price_change').round(2),
    ((pl.col('sec_price') / pl.col('sec_price').first().over('tr_id') - 1) * 100).fill_null(0).round(2).alias('roll_price_change').round(2),
    pl.col('tr_id').count().over('tr_id').alias('tr_duration_qtr')
])

df = df.drop('prev_sec_price')
df = df.with_columns([pl.col('roll_price_change').last().over('tr_id').alias('tr_price_change')])


df

cusip_ticker,tr_id,rdate,tr_type,sec_price,tr_shares,position,cost_basis,price_change,roll_price_change,tr_duration_qtr,tr_price_change
str,str,str,str,f64,i64,i64,f64,f64,f64,u32,f64
"""ACN""","""TR-TICKER-2230…","""2018-03-31T00:…","""OPEN""",153.498127,133500,133500,2.0492e7,0.0,0.0,14,79.97
"""ACN""","""TR-TICKER-2230…","""2018-06-30T00:…","""ADD""",163.589581,23900,157400,2.4402e7,6.57,6.57,14,79.97
"""ACN""","""TR-TICKER-2230…","""2018-09-30T00:…","""HOLD""",170.19695,0,157400,2.4402e7,4.04,10.88,14,79.97
"""ACN""","""TR-TICKER-2230…","""2018-12-31T00:…","""REDUCE""",141.006803,-10400,147000,2.2935e7,-17.15,-8.14,14,79.97
"""ACN""","""TR-TICKER-2230…","""2019-03-31T00:…","""HOLD""",176.020408,0,147000,2.2935e7,24.83,14.67,14,79.97
"""ACN""","""TR-TICKER-2230…","""2019-06-30T00:…","""REDUCE""",184.770932,-20400,126600,1.9166e7,4.97,20.37,14,79.97
"""ACN""","""TR-TICKER-2230…","""2019-09-30T00:…","""ADD""",192.353389,4700,131300,2.0070e7,4.1,25.31,14,79.97
"""ACN""","""TR-TICKER-2230…","""2019-12-31T00:…","""REDUCE""",210.572233,-24700,106600,1.4869e7,9.47,37.18,14,79.97
"""ACN""","""TR-TICKER-2230…","""2020-03-31T00:…","""REDUCE""",163.265823,-67100,39500,3.9138e6,-22.47,6.36,14,79.97


In [5]:
import pandas as pd
import numpy as np

raw_data = {'Date':  ['04-23-2020', '05-05-2020', '05-05-2020', '05-11-2020', '05-11-2020', 
              '05-12-2020', '05-12-2020', '05-27-2020', '06-03-2020', '06-03-2020', 
              '06-03-2020', '06-03-2020', '06-03-2020', '06-05-2020'],
    'Type': ['Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 
             'Sell', 'Sell', 'Sell', 'Buy', 'Sell', 'Sell', 'Buy'],
    'Symbol': ['TSE:AC', 'TSE:AC', 'TSE:HEXO', 'TSE:BPY.UN', 'TSE:BPY.UN', 
               'TSE:BPY.UN', 'TSE:AC', 'TSE:BPY.UN', 'TSE:AC', 'TSE:BPY.UN', 
               'TSE:AC', 'TSE:BPY.UN', 'TSE:HEXO', 'TSE:HEXO'],
    'Quantity': [75, 100, 1450, 200, 50, 80, 150, 100, 125, 100, 100, 50, 1450, 3000],
    'Price per unit': [18.04, 17.29, 0.73, 13.04, 13.06, 12.65, 15.9, 15.01, 
                        18.05, 14.75, 15.8, 14.7, 1.07, 2.50],
    'Turnover': [1353, 1729, 1058.5, 2608, 653, 1012, 2385, 1501, 2256.25, 1475, 1580, 735, 1551.5, 7500],
    }
df = pd.DataFrame (raw_data, columns = ['Date','Type','Symbol','Quantity','Price per unit', 'Turnover']).sort_values(['Date','Symbol']).reset_index(drop = True)

df.sort_values(['Symbol','Date','Type'], ascending=[True, True, True], inplace=True)
# your code
df['Adjusted Quantity'] = df.apply(lambda x: ((x.Type == "Buy") - (x.Type == "Sell")) * x['Quantity'], axis = 1)
df['Adjusted Quantity'] = df.groupby('Symbol')['Adjusted Quantity'].cumsum()
df['Adjusted Price Per Unit'] = df.apply(lambda x: ((x.Type == "Buy") - (x.Type == "Sell")) * x['Turnover'], axis = 1)
df['Adjusted Price Per Unit'] = df.groupby('Symbol')['Adjusted Price Per Unit'].cumsum().div(df['Adjusted Quantity'])

df.loc[df['Type'] == 'Sell',['Adjusted Price Per Unit']] = np.NaN
df.fillna(method='ffill', inplace=True)

# def calculate_adjusted_price_per_unit(group):
#     group['Adjusted Turnover'] = group.apply(lambda x: ((x.Type == "Buy") - (x.Type == "Sell")) * x['Turnover'], axis = 1)
#     group['Adjusted Price Per Unit'] = 0
    
#     current_turnover = 0
#     for index, row in group.iterrows():
#         current_turnover += row['Adjusted Turnover']
        
#         if row['Type'] == 'Buy':
#             group.at[index, 'Adjusted Price Per Unit'] = current_turnover / row['Adjusted Quantity']
        
#         if row['Adjusted Quantity'] == 0:
#             current_turnover = 0
    
#     return group

# df = df.groupby('Symbol').apply(calculate_adjusted_price_per_unit)
# df.drop(columns=['Adjusted Turnover'], inplace=True)


In [6]:
df

,Date,Type,Symbol,Quantity,Price per unit,Turnover,Adjusted Quantity,Adjusted Price Per Unit
0,04-23-2020,Buy,TSE:AC,75,18.04,1353.00,75,18.040000
1,05-05-2020,Buy,TSE:AC,100,17.29,1729.00,175,17.611429
5,05-12-2020,Buy,TSE:AC,150,15.90,2385.00,325,16.821538
9,06-03-2020,Buy,TSE:AC,100,15.80,1580.00,425,16.581176
8,06-03-2020,Sell,TSE:AC,125,18.05,2256.25,300,16.581176
3,05-11-2020,Buy,TSE:BPY.UN,200,13.04,2608.00,200,13.040000
4,05-11-2020,Buy,TSE:BPY.UN,50,13.06,653.00,250,13.044000
6,05-12-2020,Buy,TSE:BPY.UN,80,12.65,1012.00,330,12.948485
7,05-27-2020,Sell,TSE:BPY.UN,100,15.01,1501.00,230,12.948485
10,06-03-2020,Sell,TSE:BPY.UN,100,14.75,1475.00,130,12.948485


In [7]:
import yfinance as yf

ticker = 'ACN'
start_date = '2018-03-31'
end_date = '2023-06-30'

data = yf.download(ticker, start=start_date, end=end_date, interval='3mo')[['Close']].round(2)
quarterly_data = data.resample('Q').last().head()

display(quarterly_data)

[*********************100%%**********************]  1 of 1 completed


,Close
Date,
2018-03-31,155.74
2018-06-30,169.07
2018-09-30,164.52
2018-12-31,161.38
2019-03-31,178.07


In [8]:
returns = quarterly_data.Close.pct_change()
returns.tail()

Date
2018-03-31         NaN
2018-06-30    0.085591
2018-09-30   -0.026912
2018-12-31   -0.019086
2019-03-31    0.103420
Freq: Q-DEC, Name: Close, dtype: float64

In [9]:
import yfinance as yf
import pyfolio as pf
import pandas as pd
import numpy as np
import gzip
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

fb = yf.Ticker('ACN')
history = fb.history('max')
# history.index = history.index.tz_localize('utc')
history.tail()

ModuleNotFoundError: No module named 'pyfolio'

In [ ]:
transactions = pd.read_csv('test_txn_bak.csv',
                    index_col=0, parse_dates=True)
positions = pd.read_csv('test_pos_bak.csv',
                    index_col=0, parse_dates=True)
returns = pd.read_csv('test_returns_bak.csv',
                      index_col=0, parse_dates=True, header=None)[1]

In [ ]:
sect_map = {'COST': 'Consumer Goods', 'INTC':'Technology', 'CERN':'Healthcare', 'GPS':'Technology',
            'MMM': 'Construction', 'DELL': 'Technology', 'AMD':'Technology'}

In [ ]:
# pf.create_round_trip_tear_sheet(returns, positions, transactions)

pf.create_full_tear_sheet(returns, positions, transactions)
# # pf.create_position_tear_sheet(returns, positions, transactions)
# pf.create_perf_attrib_tear_sheet(returns, positions)
# pf.round_trips()